In [3]:
# Install required libraries
!pip install datasets transformers torch execnet


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install torch
!conda install pytorch torchvision torchaudio -c pytorch


Defaulting to user installation because normal site-packages is not writeable


In [4]:
from datasets import load_dataset
from transformers import T5ForConditionalGeneration, AutoTokenizer
# import torch
import execnet
import subprocess
import json
import traceback

# Load HumanEval dataset
dataset = load_dataset("D:\DDrive\Proposed\human-eval")

# Set up the model and tokenizer
mycheckpoint = "D:\DDrive\Proposed\Final_Checkpoint\pytorch_model.bin"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(mycheckpoint)
model = T5ForConditionalGeneration.from_pretrained(mycheckpoint).to(device)

ModuleNotFoundError: No module named 'execnet'

In [6]:
def execute_code(code_str, test_cases):
    # Function to execute the generated code and compare with expected results
    try:
        # Set up a restricted environment
        gw = execnet.makegateway("popen//python=python3")  # Adjust if needed
        ch = gw.remote_exec("""
            import sys
            code_str = channel.receive()
            exec(code_str)
        """)
        
        ch.send(code_str)


SyntaxError: incomplete input (2169659892.py, line 12)

In [ ]:
 # Check test cases
        for test_case in test_cases:
            exec_output = eval(test_case['test_input'])
            expected_output = test_case['expected_output']

            if exec_output != expected_output:
                return False, f"Expected {expected_output}, but got {exec_output}"

        return True, "All test cases passed"
    except Exception as e:
        # Capture and return error messages
        return False, str(e)

In [ ]:
# Generate code and evaluate for each problem
results = []
for example in dataset['test']:
    problem = example['prompt']
    inputs = tokenizer.encode(problem, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=100)  # Adjust max_length if needed
    
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Prepare test cases and expected outputs
    test_cases = example['test_cases']
    
    # Execute the generated code and evaluate
    success, message = execute_code(generated_code, test_cases)

    results.append({
        "problem": problem,
        "generated_code": generated_code,
        "success": success,
        "message": message
    })

# Output the evaluation results
for result in results:
    print("Problem:", result['problem'])
    print("Generated code:", result['generated_code'])
    print("Result:", "Passed" if result['success'] else "Failed")
    print("Details:", result['message'])
    print("---")